<a href="https://colab.research.google.com/github/AnkitRajSri/StockPriceForecasting/blob/master/Stock_Price_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing the libraries

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.express as ex

In [2]:
tf.__version__

'2.2.0'

### Data Preprocessing

##### Importing the training set

In [3]:
google_stock_train = pd.read_csv('Google_Stock_Price_Train.csv')
google_stock_train.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [4]:
training_set = google_stock_train.loc[:, ['Open']].values
training_set

array([[325.25],
       [331.27],
       [329.83],
       ...,
       [793.7 ],
       [783.33],
       [782.75]])

##### Feature Scaling

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_training_set = scaler.fit_transform(training_set)
scaled_training_set

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

##### Creating a data structure with 60 timesteps and 1 output

In [6]:
X_train = []
y_train = []

for i in range(60, len(scaled_training_set)):
  X_train.append(scaled_training_set[i-60:i, 0])
  y_train.append(scaled_training_set[i, 0])

X_train = np.array(X_train)
y_train = np.array(y_train)
X_train.shape

(1198, 60)

##### Reshaping the data

In [0]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [8]:
X_train.shape

(1198, 60, 1)

### Building the RNN

##### Importing the keras libraries and packages

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


##### Initializing the RNN

In [0]:
rnn = Sequential()

##### Adding the first LSTM layer and some Dropout regularization

In [0]:
rnn.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
rnn.add(Dropout(0.20))

##### Adding the other LSTM layers with some Dropout regularizations

In [0]:
rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.20))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.20))

rnn.add(LSTM(units=50))
rnn.add(Dropout(0.20))

##### Adding the output layer

In [0]:
rnn.add(Dense(units=1))

##### Compiling the RNN

In [0]:
rnn.compile(optimizer='adam', loss='mean_squared_error')

##### Training the RNN

In [15]:
rnn.fit(x=X_train, y=y_train, epochs=100, batch_size=32)

Epoch 1/100
1198/1198 [==============================] - 8s 6ms/step - loss: 0.0364
Epoch 2/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0065
Epoch 3/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0064
Epoch 4/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0056
Epoch 5/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0050
Epoch 6/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0045
Epoch 7/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0051
Epoch 8/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0044
Epoch 9/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0044
Epoch 10/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0044
Epoch 11/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0049
Epoch 12/100
1198/1198 [==============================] - 6s 5ms/step - lo

### Making the predictions and evaluating the results

##### Getting the real stock price of 2017

In [16]:
google_stock_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = google_stock_test.loc[:, ['Open']].values
real_stock_price

array([[778.81],
       [788.36],
       [786.08],
       [795.26],
       [806.4 ],
       [807.86],
       [805.  ],
       [807.14],
       [807.48],
       [807.08],
       [805.81],
       [805.12],
       [806.91],
       [807.25],
       [822.3 ],
       [829.62],
       [837.81],
       [834.71],
       [814.66],
       [796.86]])

##### Getting the predicted stock price of 2017

In [17]:
total_stock_data = pd.concat((google_stock_train['Open'], google_stock_test['Open']), axis=0)
inputs = total_stock_data[len(total_stock_data)-len(real_stock_price)-60:].values
inputs

array([779.  , 779.66, 777.71, 786.66, 783.76, 781.22, 781.65, 779.8 ,
       787.85, 798.24, 803.3 , 795.  , 804.9 , 816.68, 806.34, 801.  ,
       808.35, 795.47, 782.89, 778.2 , 767.25, 750.66, 774.5 , 783.4 ,
       779.94, 791.17, 756.54, 755.6 , 746.97, 755.2 , 766.92, 771.37,
       762.61, 772.63, 767.73, 764.26, 760.  , 771.53, 770.07, 757.44,
       744.59, 757.71, 764.73, 761.  , 772.48, 780.  , 785.04, 793.9 ,
       797.4 , 797.34, 800.4 , 790.22, 796.76, 795.84, 792.36, 790.9 ,
       790.68, 793.7 , 783.33, 782.75, 778.81, 788.36, 786.08, 795.26,
       806.4 , 807.86, 805.  , 807.14, 807.48, 807.08, 805.81, 805.12,
       806.91, 807.25, 822.3 , 829.62, 837.81, 834.71, 814.66, 796.86])

In [18]:
inputs = inputs.reshape(-1, 1)
inputs[:10]

array([[779.  ],
       [779.66],
       [777.71],
       [786.66],
       [783.76],
       [781.22],
       [781.65],
       [779.8 ],
       [787.85],
       [798.24]])

In [19]:
inputs = scaler.transform(inputs)
inputs

array([[0.9299055 ],
       [0.93113327],
       [0.92750577],
       [0.94415507],
       [0.93876032],
       [0.93403527],
       [0.93483518],
       [0.9313937 ],
       [0.94636878],
       [0.96569685],
       [0.97510976],
       [0.95966962],
       [0.97808617],
       [1.        ],
       [0.98076494],
       [0.97083116],
       [0.98450406],
       [0.96054394],
       [0.9371419 ],
       [0.92841729],
       [0.90804747],
       [0.8771858 ],
       [0.92153434],
       [0.93809063],
       [0.93165414],
       [0.95254483],
       [0.88812412],
       [0.88637547],
       [0.87032145],
       [0.88563137],
       [0.90743359],
       [0.91571173],
       [0.89941588],
       [0.91805566],
       [0.9089404 ],
       [0.9024853 ],
       [0.89456061],
       [0.91600938],
       [0.9132934 ],
       [0.88979835],
       [0.86589404],
       [0.89030062],
       [0.90335962],
       [0.89642086],
       [0.91777662],
       [0.93176576],
       [0.94114145],
       [0.957

In [0]:
X_test = []
for i in range(60, len(inputs)):
  X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)

In [21]:
X_test.shape

(20, 60)

In [22]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test.shape

(20, 60, 1)

In [23]:
predicted_stock_price = rnn.predict(X_test)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)
predicted_stock_price

array([[795.679  ],
       [792.4746 ],
       [792.5234 ],
       [793.994  ],
       [797.4069 ],
       [803.25446],
       [808.43427],
       [810.1625 ],
       [809.93677],
       [809.4295 ],
       [809.40515],
       [809.6721 ],
       [810.0037 ],
       [810.8537 ],
       [811.9322 ],
       [816.64746],
       [823.7715 ],
       [831.295  ],
       [835.07025],
       [830.24207]], dtype=float32)

##### Visualizing the result

In [24]:
import plotly.graph_objects as go

google_stock_test['Predicted Open'] = predicted_stock_price

fig = go.Figure(data=[go.Line(x=google_stock_test['Date'], y=google_stock_test['Open'], name='Real Stock Price'),
                      go.Line(x=google_stock_test['Date'], y=google_stock_test['Predicted Open'], name='Predicted Stock Price')], )
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### Evaluating the Root Mean Squared Error

In [25]:
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))
rmse

11.293474409400584